In [2]:
%pylab inline
from pulp import *

Populating the interactive namespace from numpy and matplotlib


__1. Formulate and solve with Lindo the critical path method handout attached__



![](http://snag.gy/AEUdG.jpg)

Definie the variable $t_i$ to be the mimimum time spent at node $i$ where all activities preceding node $i$ have been completed

The critical path is FD(8) --> CS(5) --> 1B(3) --> 2FL(3) --> 2I(5) --> 2F(10) is the critical path

In [3]:
Nodes = ["FD","CS","1I","1B","1F","2FL","2I","2B","2F","RF"]
Cost = [8,5,4,3,12,3,5,4,10,1]
c = makeDict([Nodes],Cost,0)
E =   [["FD","CS"],
       ["CS","1I"],
       ["CS","1B"],
       ["1B","1F"],
       ["1I","1F"],
       ["1B","2FL"],
       ["2FL","2I"],
       ["2FL","2B"],
       ["2B","2F"],
       ["2B","RF"],
       ["2I","2F"],
       ["RF","2F"]]
# declare variables
t = LpVariable.dicts("t",Nodes, lowBound=0)
prob = LpProblem("minimum time usd to finish constructing",LpMinimize)
prob += t['2F']+10
# decalre constraints
prob += t["FD"] == 0
for (i,j) in E:
    prob += t[j] - t[i] >= c[i], "Node %s to Node %s work at least %s"%(i,j,c[i])
prob.solve()
print "Minimum time usd to finish constructing is %s" %(value(prob.objective))
for v in prob.variables():
    print v.name, " = ", v.varValue
print "Status:", LpStatus[prob.status]
prob

Minimum time usd to finish constructing is 34.0
t_1B  =  13.0
t_1F  =  17.0
t_1I  =  13.0
t_2B  =  19.0
t_2F  =  24.0
t_2FL  =  16.0
t_2I  =  19.0
t_CS  =  8.0
t_FD  =  0.0
t_RF  =  23.0
Status: Optimal


minimum time usd to finish constructing:
MINIMIZE
1*t_2F + 10
SUBJECT TO
_C1: t_FD = 0

Node_FD_to_Node_CS_work_at_least_8: t_CS - t_FD >= 8

Node_CS_to_Node_1I_work_at_least_5: t_1I - t_CS >= 5

Node_CS_to_Node_1B_work_at_least_5: t_1B - t_CS >= 5

Node_1B_to_Node_1F_work_at_least_3: - t_1B + t_1F >= 3

Node_1I_to_Node_1F_work_at_least_4: t_1F - t_1I >= 4

Node_1B_to_Node_2FL_work_at_least_3: - t_1B + t_2FL >= 3

Node_2FL_to_Node_2I_work_at_least_3: - t_2FL + t_2I >= 3

Node_2FL_to_Node_2B_work_at_least_3: t_2B - t_2FL >= 3

Node_2B_to_Node_2F_work_at_least_4: - t_2B + t_2F >= 4

Node_2B_to_Node_RF_work_at_least_4: - t_2B + t_RF >= 4

Node_2I_to_Node_2F_work_at_least_5: t_2F - t_2I >= 5

Node_RF_to_Node_2F_work_at_least_1: t_2F - t_RF >= 1

VARIABLES
t_1B Continuous
t_1F Continuous
t_1I Continuous
t_2B Continuous
t_2F Continuous
t_2FL Continuous
t_2I Continuous
t_CS Continuous
t_FD Continuous
t_RF Continuous

__2. Formulate and solve with Lindo the shortest path for the on pg 648 using s as the start node, x as end node__



shortest path cost: 9; the shortest path from s to x is through t

In [4]:
string = "s y x t z"
nodes = string.split()
cost = {("s","t"):3,
        ("s","y"):5,
        ("t","x"):6,
        ("t","y"):2,
        ("y","t"):1,
        ("y","x"):4,
        ("y","z"):6,
        ("z","s"):3,
        ("z","x"):7,
        ("x","z"):2}
var = LpVariable.dicts("edges",cost, lowBound=0,upBound=1,cat=LpInteger)
prob = LpProblem("shortest path problem", LpMinimize)
prob += lpSum([var[a]* cost[a] for a in cost]), "Total Cost of Path"
for n in nodes:
    prob += ((n=='s')+ lpSum([var[(i,j)] for (i,j) in cost if j == n]) == 
            (n=='x')+ lpSum([var[(i,j)] for (i,j) in cost if i == n])),\
    "Flow Conservation in Node %s"%n
prob.solve()
print "Shortest Path Total Cost = ", value(prob.objective)
print "the path from s to x is "
for a in cost:
    if (int(var[a].varValue) != 0):
        print a
print "Status:", LpStatus[prob.status]
prob

Shortest Path Total Cost =  9.0
the path from s to x is 
('t', 'x')
('s', 't')
Status: Optimal


shortest path problem:
MINIMIZE
3*edges_('s',_'t') + 5*edges_('s',_'y') + 6*edges_('t',_'x') + 2*edges_('t',_'y') + 2*edges_('x',_'z') + 1*edges_('y',_'t') + 4*edges_('y',_'x') + 6*edges_('y',_'z') + 3*edges_('z',_'s') + 7*edges_('z',_'x') + 0
SUBJECT TO
Flow_Conservation_in_Node_s: - edges_('s',_'t') - edges_('s',_'y')
 + edges_('z',_'s') = -1

Flow_Conservation_in_Node_y: edges_('s',_'y') + edges_('t',_'y')
 - edges_('y',_'t') - edges_('y',_'x') - edges_('y',_'z') = 0

Flow_Conservation_in_Node_x: edges_('t',_'x') - edges_('x',_'z')
 + edges_('y',_'x') + edges_('z',_'x') = 1

Flow_Conservation_in_Node_t: edges_('s',_'t') - edges_('t',_'x')
 - edges_('t',_'y') + edges_('y',_'t') = 0

Flow_Conservation_in_Node_z: edges_('x',_'z') + edges_('y',_'z')
 - edges_('z',_'s') - edges_('z',_'x') = 0

VARIABLES
0 <= edges_('s',_'t') <= 1 Integer
0 <= edges_('s',_'y') <= 1 Integer
0 <= edges_('t',_'x') <= 1 Integer
0 <= edges_('t',_'y') <= 1 Integer
0 <= edges_('x',_'z') <= 1 Integer
0 <= edges_(

__3. Formulate to solve the coloring problem on the graph quiz (assume at most 6 colors), solve with lindo__



![](http://snag.gy/qCNdq.jpg)

minimum color used: 3

In [5]:
nc = 6 #max number of color
n = 11 #number of nodes
V = range(1,n+1) #set of nodes
color = range(1,nc+1) #set of color
E =   [[1, 2],
       [1, 5],
       [2, 4],
       [2, 5],
       [3, 6],
       [3, 8],
       [3, 7],
       [3, 4],
       [4, 5],
       [4, 6],
       [4, 7],
       [5, 7],
       [6, 8],
       [7, 8],
       [7, 9],
       [8, 10],
       [10, 11]]
# vertically ordered nodes len(E) == 17
# declare variables
# x[i,c] = 1 means that node i is assigned color c
x = LpVariable.dicts("x",(V,color), lowBound=0,upBound=1,cat=LpInteger)
all_possible_x = [(s,t) for s in V for t in color]
# y[c] = 1 means that color c is used, i.e. assigned to some node
y = LpVariable.dicts("y",color,0,1,LpInteger)
prob = LpProblem("minimum number of colors used",LpMinimize)
prob += lpSum([y[c] for c in color])
# each node must be assigned exactly one color
for i in V:
    prob += lpSum([x[i][c] for c in color]) == 1, "Node %s has been assigned one color"%i
    # adjacent nodes cannot be assigned the same color
# because the index start with 0, so I have to do i-1
for (i,j) in E:
    for c in color:
        prob += x[i][c] + x[j][c] <= 1,"color %s isn't used in adjacent node %s and %s" %(c,i,j)
        # make sure color is used 
for c in color:
    for i in V:
        prob += x[i][c]<= y[c]
prob.solve()
print "Minimum number of color used = %s" %(value(prob.objective))
for v in V:
    print 'node %s' % v, 'use color',
    for c in color:
        if int(x[v][c].varValue) ==1:
            print c

print "Status:", LpStatus[prob.status]

Minimum number of color used = 3.0
node 1 use color 2
node 2 use color 5
node 3 use color 6
node 4 use color 2
node 5 use color 6
node 6 use color 5
node 7 use color 5
node 8 use color 2
node 9 use color 6
node 10 use color 6
node 11 use color 2
Status: Optimal


__4. We defined two grammars on page 3 and page 7. Determine the type of classification for each grammar__

1. grammar in page 3 is type 2 becauselLeft hand side of each production is a single, nonterminal symbol and the right hand side consists of one or more terminal or nonterminal symbols (e.g., verbphrase $\rightarrow$ (verb+adverb), and they are all nonterminate symbols, violating type 3).

2. grammar in page 7 is type 2 becauselLeft hand side of each production is a single, nonterminal symbol and the right hand side consists of one or more terminal or nonterminal symbols (e.g., S $\rightarrow$ (S+F) violates type 3).

__5 Discuss the differences between LL and LR parsings.  Give two real-world examples for LL and LR__



first real example: traversal of binary tree
![](http://i.stack.imgur.com/QLYBe.png)

LL: top-down parsing, find the leftmost(top most) symbol first and then parse level by level.
    the output: + 1 * 2 3    (## pre-order traversal)
 
 
LR: bottom-up parsing, find the rightmost(nearest) symbol first and then work its way upward.
    the output: 1 2 3 * +    (## post-order traversal)


second real example: Dangling else

> if a then if b then s else s2 can be parsed in two different ways:
>> if a then (if b then s) else s2 $\tag{1}$
>> if a then (if b then s else s2) $\tag{2}$

LL: top-down parsing, find the leftmost(top most) symbol first and then parse level by level.
    Will choose the (1) since the first if is the top most symbols
 
 
LR: bottom-up parsing, find the rightmost(nearest) symbol first and then work its way upward.
    Will choose the (2) since it associate else with the nearest if.


__6. Write out explicitly the linear program corresponding to finding the maximum flow in Figure 26.1(a).__

![](http://snag.gy/hr6HE.jpg)

the maxflow is 23
vancouver to edmonton:  10.0
vancouver to calgary:   13.0
edmonton  to saskatoon: 12.0
calgary to edmonton:    2.0
calgary to regina:      11.0
saskatoon to winnipeg:  19.0
saskatoon to calgary:   0.0
regina to winnipeg:     4.0
regina to saskatoon:    7.0

In [6]:
string = "s v1 v2 v3 v4 t"
nodes = string.split()
capacity = {("s","v1"):16,
        ("s","v2"):13,
        ("v1","v3"):12,
        ("v2","v1"):4,
        ("v2","v4"):14,
        ("v3","v2"):9,
        ("v3","t"):20,
        ("v4","v3"):7,
        ("v4","t"):4}
var = LpVariable.dicts("edges",capacity, lowBound=0,cat=LpContinuous)
prob = LpProblem("max flow problem", LpMaximize)
prob += lpSum([var[(i,j)] for (i,j) in var if i == 's']), "Total Cost of Path"
# constraint 1: capacity
for a in capacity:
    prob += var[a] <= capacity[a]
# constraint 2: in == out
for n in nodes: 
    if not n.startswith('s') and not n.startswith('t'):
        prob += (lpSum([var[(i,j)] for (i,j) in var if j == n]) == 
                 lpSum([var[(i,j)] for (i,j) in var if i == n])),\
        "Node %s"%n
prob.solve()
print("objective=", value(prob.objective))
for v in prob.variables():
    print(v.name, "=", v.varValue)
print "Status:", LpStatus[prob.status]
prob

('objective=', 23.0)
("edges_('s',_'v1')", '=', 10.0)
("edges_('s',_'v2')", '=', 13.0)
("edges_('v1',_'v3')", '=', 12.0)
("edges_('v2',_'v1')", '=', 2.0)
("edges_('v2',_'v4')", '=', 11.0)
("edges_('v3',_'t')", '=', 19.0)
("edges_('v3',_'v2')", '=', 0.0)
("edges_('v4',_'t')", '=', 4.0)
("edges_('v4',_'v3')", '=', 7.0)
Status: Optimal


max flow problem:
MAXIMIZE
1*edges_('s',_'v1') + 1*edges_('s',_'v2') + 0
SUBJECT TO
_C1: edges_('v3',_'v2') <= 9

_C2: edges_('v1',_'v3') <= 12

_C3: edges_('v2',_'v1') <= 4

_C4: edges_('v4',_'v3') <= 7

_C5: edges_('s',_'v2') <= 13

_C6: edges_('v4',_'t') <= 4

_C7: edges_('v2',_'v4') <= 14

_C8: edges_('s',_'v1') <= 16

_C9: edges_('v3',_'t') <= 20

Node_v1: edges_('s',_'v1') - edges_('v1',_'v3') + edges_('v2',_'v1') = 0

Node_v2: edges_('s',_'v2') - edges_('v2',_'v1') - edges_('v2',_'v4')
 + edges_('v3',_'v2') = 0

Node_v3: edges_('v1',_'v3') - edges_('v3',_'t') - edges_('v3',_'v2')
 + edges_('v4',_'v3') = 0

Node_v4: edges_('v2',_'v4') - edges_('v4',_'t') - edges_('v4',_'v3') = 0

VARIABLES
edges_('s',_'v1') Continuous
edges_('s',_'v2') Continuous
edges_('v1',_'v3') Continuous
edges_('v2',_'v1') Continuous
edges_('v2',_'v4') Continuous
edges_('v3',_'t') Continuous
edges_('v3',_'v2') Continuous
edges_('v4',_'t') Continuous
edges_('v4',_'v3') Continuous

__7 Prove that if G is an undirected bipartite graph with an odd number of vertices, then G is nonhamiltonian.__

Suppose G is biparate graph and it is Hamiltonian. 

Since G is Hamiltonian, G has Hamiltonian cycle: we can find a cycle goes through all the vertices in G and never repeat. 

Since G is a bipartite graph (suppose left set is A and right set is B), every edge connects a vertex in A to one in B

If G has odd number of vertices, then $|A| \neq |B|$. Suppose $|A| = m \ and \ |B| = n \ and \ m > n$. Suppose we start from a node in B and after traversing $2n$ vertices, it will go back to where it starts but there would still be $m-n$ vertices left in set A that haven't be connected. Therefore, this proved that G cannot be hamltonian.

__9 Let 2-CNF-SAT be the set of satisfiable boolean formulas in CNF with exactly 2
literals per clause. Show that 2-CNF-SAT $\in$ P. Make your algorithm as efficient as
possible.__

It's done in the next homework

__10. finish the following__

 a) Find a regular expression for the set of strings over {a,b} that does not contain the substring aaa. Give the FSM.

sol: $(b \vee ab \vee aab)^∗ (a \vee aa \vee \lambda)$

b) The set of binary strings with every 1 followed by two 0s.

sol: $0^* ( 100 \vee 0 )^*$

c) The set of binary strings ending in 00 and not containing 11.

sol: $0^* ( 10 \vee 0 )^* 00$

d)  The set of binary strings containing an even number of 1s.

sol: $( 0^* 10^* 10^* )^*$